# RECOMMENDER_SYSTEM

## ABSTRACT

In this notebook we are building the recommender system by calculating the cosine similarity of the candidate skills and the skills in the job description, the estimated salary of the job and the rating of the job. We then extract the float salary and the ratings and normalize them to get the value between 0 and 1. Then we next take the weighted score of similarity_score(50%), Normalized_salary(30%) and Normalized_rating(20%) and calculate the total of this. We then recommend the candidates the job posting in descending order of total_score.

**WHAT IS A RECOMMENDER SYSTEM?**

A recommendation system is a computer program that helps a user discover products and content by predicting the user’s rating of each item and showing them the items that they would rate highly. Recommendation systems are everywhere. If you’ve ever looked for books on Amazon or browsed through posts on Facebook, you’ve used the recommendation system without even knowing it. With online shopping, consumers have nearly infinite choices. No one has enough time to try every product for sale. Recommendation systems play an important role in helping users find products and content they care about.

![](files/ss/recom.png)

![](files/ss/rec.jpg)

**WHAT ARE THE DIFFERENT TYPES OF RECOMMENDATIONS?**

There are basically three important types of recommendation engines:

**Collaborative filtering**

**Content-Based Filtering**

**Hybrid Recommendation Systems**

**Collaborative filtering:**

This filtering method is usually based on collecting and analyzing information on user’s behaviors, their activities or preferences and predicting what they will like based on the similarity with other users. A key advantage of the collaborative filtering approach is that it does not rely on machine analyzable content and thus it is capable of accurately recommending complex items such as movies without requiring an “understanding” of the item itself. Collaborative filtering is based on the assumption that people who agreed in the past will agree in the future, and that they will like similar kinds of items as they liked in the past. For example, if a person A likes item 1, 2, 3 and B like 2,3,4 then they have similar interests and A should like item 4 and B should like item 1.

Further, there are several types of collaborative filtering algorithms:

**User-User Collaborative Filtering: **

Here, we try to search for lookalike customers and offer products based on what his/her lookalike has chosen. This algorithm is very effective but takes a lot of time and resources. This type of filtering requires computing every customer pair information which takes time. So, for big base platforms, this algorithm is hard to put in place.
Item-Item Collaborative Filtering: It is very similar to the previous algorithm, but instead of finding a customer look alike, we try finding item look alike. Once we have item look alike matrix, we can easily recommend alike items to a customer who has purchased any item from the store. This algorithm requires far fewer resources than user-user collaborative filtering. Hence, for a new customer, the algorithm takes far lesser time than user-user collaborate as we don’t need all similarity scores between customers. Amazon uses this approach in its recommendation engine to show related products which boost sales.
Other simpler algorithms: There are other approaches like market basket analysis, which generally do not have high predictive power than the algorithms described above.

**Content-based filtering:**

These filtering methods are based on the description of an item and a profile of the user’s preferred choices. In a content-based recommendation system, keywords are used to describe the items; besides, a user profile is built to state the type of item this user likes. In other words, the algorithms try to recommend products which are similar to the ones that a user has liked in the past. The idea of content-based filtering is that if you like an item you will also like a ‘similar’ item. For example, when we are recommending the same kind of item like a movie or song recommendation. This approach has its roots in information retrieval and information filtering research.

A major issue with content-based filtering is whether the system is able to learn user preferences from users actions about one content source and replicate them across other different content types. When the system is limited to recommending the content of the same type as the user is already using, the value from the recommendation system is significantly less when other content types from other services can be recommended. For example, recommending news articles based on browsing of news is useful, but wouldn’t it be much more useful when music, videos from different services can be recommended based on the news browsing.

![](files/ss/type_recommend.png)

**Hybrid Recommendation systems:**

Recent research shows that combining collaborative and content-based recommendation can be more effective. Hybrid approaches can be implemented by making content-based and collaborative-based predictions separately and then combining them. Further, by adding content-based capabilities to a collaborative-based approach and vice versa; or by unifying the approaches into one model.

Several studies focused on comparing the performance of the hybrid with the pure collaborative and content-based methods and demonstrate that hybrid methods can provide more accurate recommendations than pure approaches. Such methods can be used to overcome the common problems in recommendation systems such as cold start and the data paucity problem.

Netflix is a good example of the use of hybrid recommender systems. The website makes recommendations by comparing the watching and searching habits of similar users (i.e., collaborative filtering) as well as by offering movies that share characteristics with films that a user has rated highly (content-based filtering).

![](files/ss/hybrid.jpg)

In [1]:
#importing all the important libraries
import docx, os
import pandas as pd
from ast import literal_eval

from collections import Counter

import math

import re

In [2]:
#creating a dataframe
recommend=pd.DataFrame(columns=['Name','Skills','Title','Company_Name','Company_Location','Salary','Company_Rating','Similarity_score'],index=(range(10000)))

In [3]:
#reading the csv files
comp=pd.read_csv('glassdor_scraping.csv')
res=pd.read_csv('resume_parser.csv')

In [4]:
#displaying the first 5 job postings
comp.head(5)

,Title,Company,Location,Description,Salary,Company_Rating,Skillset
0,Senior Data Scientist,Instacart,"– San Francisco, CA","Founded in 2012, Instacart is a leader in Nort...",$144K-$193K (Glassdoor est.),3.7 ★,"['python', 'sql', 'dashboards', 'leader']"
1,DATA SCIENTIST / ANALYTIC CONSULTANT 4,Wells Fargo,"– San Francisco, CA","Job Description\r\n\r\nAt Wells Fargo, we want...",$82K-$131K (Glassdoor est.),3.5 ★,"['python', 'hive', 'tensorflow', 'keras', 'mac..."
2,Sr. Applications Scientist – Charged Particle ...,Multibeam,"– Santa Clara, CA",Sr. Applications Scientist – Charged Particle ...,Employer Provided Salary:$100K-$135K,5.0 ★,"['leader', 'data analysis']"
3,Principal Scientist,bioMérieux,"– San Diego, CA",World leader in the field of in vitro diagnost...,$91K-$126K (Glassdoor est.),3.4 ★,"['leader', 'phd', 'data analysis']"
4,Data Engineer,LeadCrunch,"– San Diego, CA",Data Engineer\r\n\r\nAre you ready to be a par...,Employer Provided Salary:$125K-$155K,4.1 ★,"['python', 'hadoop', 'leader', 'data mining']"


In [5]:
#displaying the first 5 resume
res.head(5)

,Name,Skills
0,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen..."
1,POOJITH SHANKAR SHETTY,"['sql', 'oracle', 'java', 'aws', 'nosql', 'mon..."
2,Prashant Reddy,"['python', 'oracle', 'machine learning', 'big ..."
3,SHWETA PATHAK,"['big data', 'data mining', 'sql', 'oracle', '..."


**WHAT IS COSINE SIMILARITY?**

Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0,π] radians. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude. The cosine similarity is particularly used in positive space, where the outcome is neatly bounded in [0,1]. The name derives from the term "direction cosine": in this case, unit vectors are maximally "similar" if they're parallel and maximally "dissimilar" if they're orthogonal (perpendicular). This is analogous to the cosine, which is unity (maximum value) when the segments subtend a zero angle and zero (uncorrelated) when the segments are perpendicular.

![](files/ss/cos-1.png)

![](files/ss/cos-2.png)

**WHEN TO USE COSINE SIMILARITY?**

Cosine similarity is generally used as a metric for measuring distance when the magnitude of the vectors does not matter. This happens for example when working with text data represented by word counts. We could assume that when a word (e.g. science) occurs more frequent in document 1 than it does in document 2, that document 1 is more related to the topic of science. However, it could also be the case that we are working with documents of uneven lengths (Wikipedia articles for example). Then, science probably occurred more in document 1 just because it was way longer than document 2. Cosine similarity corrects for this.

Text data is the most typical example for when to use this metric. However, you might also want to apply cosine similarity for other cases where some properties of the instances make so that the weights might be larger without meaning anything different. Sensor values that were captured in various lengths (in time) between instances could be such an example.

https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists

In this method we calculate the cosine similarity between two lists. First create the union of two lists. Then calculate the sum of dot product between the two lists. Then calculate the sqrt of each list and return the distance by finding the dotprod/(magA * magB)

In [6]:
#define a method to calculate the consine similarity
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

Calculate the cosine_similarity distance for all the jobs postings for each candidates resume's skills.

In [7]:
#find the cosine_similarity for all the rows in the dataframe
j=0
k=0
while(k<len(res)):
    if(res['Name'][k] !='N'):
        i=0
        while(i<len(comp['Skillset'])):
            counterA = Counter(comp['Skillset'][i])
            counterB = Counter(res['Skills'][k])
            sim = counter_cosine_similarity(counterA,counterB)
            recommend['Name'][j] = res['Name'][k]
            recommend['Skills'][j] = res['Skills'][k]
            recommend['Company_Name'][j] = comp['Company'][i]
            recommend['Company_Location'][j] = comp['Location'][i]
            recommend['Salary'][j] = comp['Salary'][i]
            recommend['Company_Rating'][j] = comp['Company_Rating'][i]
            recommend['Similarity_score'][j] = sim
            recommend['Title'][j]=comp['Title'][i]
            i=i+1
            j=j+1
        k=k+1
    else:
        k=k+1

In [8]:
recommend

,Name,Skills,Title,Company_Name,Company_Location,Salary,Company_Rating,Similarity_score
0,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Senior Data Scientist,Instacart,"– San Francisco, CA",$144K-$193K (Glassdoor est.),3.7 ★,0.927982
1,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",DATA SCIENTIST / ANALYTIC CONSULTANT 4,Wells Fargo,"– San Francisco, CA",$82K-$131K (Glassdoor est.),3.5 ★,0.931434
2,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Sr. Applications Scientist – Charged Particle ...,Multibeam,"– Santa Clara, CA",Employer Provided Salary:$100K-$135K,5.0 ★,0.861279
3,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Principal Scientist,bioMérieux,"– San Diego, CA",$91K-$126K (Glassdoor est.),3.4 ★,0.913769
4,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Data Engineer,LeadCrunch,"– San Diego, CA",Employer Provided Salary:$125K-$155K,4.1 ★,0.905892
5,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Advanced Analytics Manager - Healthcare,Central California Alliance for Health,"– Scotts Valley, CA",$113K-$155K (Glassdoor est.),3.5 ★,0.907355
6,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...","Senior Software Engineer, Data Infrastructure",New Relic,"– San Francisco, CA, United States",$138K-$221K (Glassdoor est.),4.5 ★,0.938659
7,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...",Financial Analytics Manager,Central California Alliance for Health,"– Scotts Valley, CA",$113K-$155K (Glassdoor est.),3.5 ★,0.881293
8,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...","Senior Scientist, Culture Process Development",Roche,– Pleasanton,$81K-$122K (Glassdoor est.),3.9 ★,0.028341
9,AYUSH JAIN,"['mongodb', 'sql', 'oracle', 'java', 'qlik sen...","Data Analyst, Business Intelligence","Gemological Institute of America, Inc.","– Carlsbad, CA",$40K-$64K (Glassdoor est.),2.9 ★,0.918583


In [9]:
job=pd.DataFrame(columns=['Candidate_Name','Job_Title','Company_Name','Company_Location','Estimated_Salary','Company_Rating','Similarity_score'],index=(range(10000)))

We filter the results by selecting only the results which have the similarity score greater than 0.5. 

In [10]:
#creating a dataframe with rows having the cosine_similarity greater than 0.5
i=0
j=0
while(i<len(recommend)):
    if(recommend['Similarity_score'][i]>0.5):
        job['Candidate_Name'][j]=recommend['Name'][i]
        job['Company_Name'][j]=recommend['Company_Name'][i]
        job['Company_Location'][j]=recommend['Company_Location'][i]
        job['Estimated_Salary'][j]=recommend['Salary'][i]
        job['Company_Rating'][j]=recommend['Company_Rating'][i]
        job['Similarity_score'][j]=recommend['Similarity_score'][i]
        job['Job_Title'][j]=recommend['Title'][i]
        j=j+1
    i=i+1

In [11]:
#dropping the null values
job=job.dropna()
job

,Candidate_Name,Job_Title,Company_Name,Company_Location,Estimated_Salary,Company_Rating,Similarity_score
0,AYUSH JAIN,Senior Data Scientist,Instacart,"– San Francisco, CA",$144K-$193K (Glassdoor est.),3.7 ★,0.927982
1,AYUSH JAIN,DATA SCIENTIST / ANALYTIC CONSULTANT 4,Wells Fargo,"– San Francisco, CA",$82K-$131K (Glassdoor est.),3.5 ★,0.931434
2,AYUSH JAIN,Sr. Applications Scientist – Charged Particle ...,Multibeam,"– Santa Clara, CA",Employer Provided Salary:$100K-$135K,5.0 ★,0.861279
3,AYUSH JAIN,Principal Scientist,bioMérieux,"– San Diego, CA",$91K-$126K (Glassdoor est.),3.4 ★,0.913769
4,AYUSH JAIN,Data Engineer,LeadCrunch,"– San Diego, CA",Employer Provided Salary:$125K-$155K,4.1 ★,0.905892
5,AYUSH JAIN,Advanced Analytics Manager - Healthcare,Central California Alliance for Health,"– Scotts Valley, CA",$113K-$155K (Glassdoor est.),3.5 ★,0.907355
6,AYUSH JAIN,"Senior Software Engineer, Data Infrastructure",New Relic,"– San Francisco, CA, United States",$138K-$221K (Glassdoor est.),4.5 ★,0.938659
7,AYUSH JAIN,Financial Analytics Manager,Central California Alliance for Health,"– Scotts Valley, CA",$113K-$155K (Glassdoor est.),3.5 ★,0.881293
8,AYUSH JAIN,"Data Analyst, Business Intelligence","Gemological Institute of America, Inc.","– Carlsbad, CA",$40K-$64K (Glassdoor est.),2.9 ★,0.918583
9,AYUSH JAIN,Data Engineer,NCSOFT,"– Aliso Viejo, CA",$98K-$131K (Glassdoor est.),2.7 ★,0.959133


In [12]:
fin=pd.DataFrame(columns=['Candidate_Name','Job_Title','Company_Name','Estimated_Salary','Company_Rating','Similarity_Score'],index=(range(10000)))

**PATTERN MATCHING USING REGEX**

A regular expression is a special sequence of characters that helps you match or find other strings or sets of strings, using a specialized syntax held in a pattern. Regular expressions are widely used in UNIX world.

The module re provides full support for Perl-like regular expressions in Python. The re module raises the exception re.error if an error occurs while compiling or using a regular expression.

Some of the values in the estimated salary are given in the Per hour basis which we have converted to annualy by multiplying the value by 40(hour per week) and 52(number of weeks).

In [13]:
#extracting the salary and rating for each row, just want to extract the numbers and no text from these two columns
i=0

while(i<len(job)):
    st= job['Estimated_Salary'][i]
    string=''.join(st)
    #pattern if the salary is mentioned per hour
    pat=re.compile('.*Per Hour')
    matching=pat.findall(string)
    #this pattern is for salary listed in the per hour format
    if matching:
        mat1=''.join(matching)
        mt1=mat1.replace('$','').replace('Per Hour','')
        #spilt the value by -, the value before hifen is the lower value and the value after hifen is the upper value
        lower1,upper1=mt1.split('-')
        #converting the lower value annually
        low1=int(lower1)*40*52
        #converting the upper value annually
        upp1=int(upper1)*40*52
        #finding the average salalry
        average1=(low1+upp1)/2000
        fin.loc[i]['Estimated_Salary']=average1
        
    #this is for salary listed other than the Per hour
    else:
        #pattern if the salary is mentioned yearly
        pattern=re.compile('.*K-.*K')
        match=pattern.findall(string)
        mat=''.join(match)
        mt=mat.replace('$','').replace('K','').replace('Employer Provided Salary:','')
        #spilt the value by -, the value before hifen is the lower value and the value after hifen is the upper value
        lower, upper=mt.split('-')
        low=int(lower)
        upp=int(upper)
        #finding the average salalry
        average=(low+upp)/2
        fin.loc[i]['Estimated_Salary']=average
    
    rat=job['Company_Rating'][i]
    fin.loc[i]['Company_Rating']=rat[:3]
    fin.loc[i]['Candidate_Name']=job['Candidate_Name'][i]
    fin.loc[i]['Company_Name']=job['Company_Name'][i]
    fin.loc[i]['Similarity_Score']=job['Similarity_score'][i]
    fin.loc[i]['Job_Title']=job['Job_Title'][i]
    
    i=i+1
    

In [14]:
#dropping the null rows and converting the datatype of three columns
fin=fin.dropna()
fin['Similarity_Score']=fin['Similarity_Score'].astype(float)
fin['Company_Rating']=fin['Company_Rating'].astype(float)
fin['Estimated_Salary']=fin['Estimated_Salary'].astype(float)
fin

,Candidate_Name,Job_Title,Company_Name,Estimated_Salary,Company_Rating,Similarity_Score
0,AYUSH JAIN,Senior Data Scientist,Instacart,168.50,3.7,0.927982
1,AYUSH JAIN,DATA SCIENTIST / ANALYTIC CONSULTANT 4,Wells Fargo,106.50,3.5,0.931434
2,AYUSH JAIN,Sr. Applications Scientist – Charged Particle ...,Multibeam,117.50,5.0,0.861279
3,AYUSH JAIN,Principal Scientist,bioMérieux,108.50,3.4,0.913769
4,AYUSH JAIN,Data Engineer,LeadCrunch,140.00,4.1,0.905892
5,AYUSH JAIN,Advanced Analytics Manager - Healthcare,Central California Alliance for Health,134.00,3.5,0.907355
6,AYUSH JAIN,"Senior Software Engineer, Data Infrastructure",New Relic,179.50,4.5,0.938659
7,AYUSH JAIN,Financial Analytics Manager,Central California Alliance for Health,134.00,3.5,0.881293
8,AYUSH JAIN,"Data Analyst, Business Intelligence","Gemological Institute of America, Inc.",52.00,2.9,0.918583
9,AYUSH JAIN,Data Engineer,NCSOFT,114.50,2.7,0.959133


Next we calculate the mean, min and max of 'Estimated_Salary' and 'Company_Rating'

In [15]:
#finding the mean,min and max of salary and rating
mean_salary=fin['Estimated_Salary'].mean()
mean_rating=fin['Company_Rating'].mean()     
min_salary=fin['Estimated_Salary'].min()
min_rating=fin['Company_Rating'].min()
max_salary=fin['Estimated_Salary'].max()
max_rating=fin['Company_Rating'].max()

In [16]:
recommd=pd.DataFrame(columns=['Candidate_Name','Job_Title','Company_Name','Estimated_Salary','Company_Rating','Total_Score'],index=(range(10000)))

**WHAT IS NORMALIZATION OF VALUES?**

It is the process of bringing all the values to common scale.

![](files/ss/nor.png)

**WHY TO NORMALIZE?**

It is important to normalize because the values of similarity_score is between 0 and 1, whereas the salary value is greater than 0 and the rating is between 0 and 5. So we want these two columns also to be between 0 and 1 so that we can calculate the weighted score.

After normalizing we calculate the total score to finalize the recommendations.

**The total_score is calculated using the following weightage:**

**50%(Similarity_score) + 30%(Weighted_Salary) + 20%(Weighted_Rating)**

In [17]:
#normalizing the salary and rating to values between 0 and 1 by using (value-min(column))/(max(column)-min(column))
#calculate the total by summing the 30% of salary, 20% of rating and 50% of the similarity score
#display the result on the basis of total_score in descending order
i=0
while(i<len(fin)):
    sal=abs(fin['Estimated_Salary'][i]-min_salary)/abs(max_salary-min_salary)
    rat=abs(fin['Company_Rating'][i]-min_rating)/abs(max_rating-min_rating)
    sal=(sal*30)/100
    rat=(rat*20)/100
    sim=fin['Similarity_Score'][i]
    sim=(sim*50)/100
    total=sal+rat+sim
    recommd.loc[i]['Candidate_Name']=fin['Candidate_Name'][i]
    recommd.loc[i]['Company_Name']=fin['Company_Name'][i]
    recommd.loc[i]['Job_Title']=fin['Job_Title'][i]
    recommd.loc[i]['Estimated_Salary']=fin['Estimated_Salary'][i]
    recommd.loc[i]['Company_Rating']=fin['Company_Rating'][i]
    recommd.loc[i]['Total_Score']=total
    i=i+1
recommd.sort_values(by=['Candidate_Name','Total_Score'],ascending=False,inplace=True)
    

Display the results in descending order of total_score for each candidate.

In [18]:
recommd=recommd.dropna()
recommd=recommd.drop_duplicates(subset=['Candidate_Name','Company_Name','Job_Title'],keep='first',inplace=False)
recommd
                                        

,Candidate_Name,Job_Title,Company_Name,Estimated_Salary,Company_Rating,Total_Score
4369,SHWETA PATHAK,Senior Data Scientist,Wealthfront,205,4.4,0.87707
4511,SHWETA PATHAK,Technical Lead - Data Engineering,Grand Rounds,182,4.7,0.86489
4835,SHWETA PATHAK,"Principal Scientist, Computational Biology",Tesaro,224.5,3.8,0.851068
4460,SHWETA PATHAK,Senior Data Scientist,Intercom,203,4,0.841585
4418,SHWETA PATHAK,Lead Data Scientist,BCG Digital Ventures,183.5,4.2,0.840148
4416,SHWETA PATHAK,Senior Staff Data Scientist,Huawei Technologies,239,3.3,0.836661
4739,SHWETA PATHAK,"Senior Software Engineer, MuleSoft Integration...",New Relic,179.5,4.5,0.835385
4380,SHWETA PATHAK,Senior Applied Scientist - Machine Learning,Wealthfront,187,4.4,0.834871
4801,SHWETA PATHAK,Backend Engineer,Super Lucky Casino,175,5,0.829702
5125,SHWETA PATHAK,Director of Business Intelligence,Hopper,183.5,3.9,0.822887


## CONCLUSION 

In this notebook we have taken the csv file from scrapping data from glassdoor and parsing the resume as input. We have used cosine_similarity to find the similarity between the skills in the job description and skillset of the each candidate. Then we computed the total_score by using the weighted total of 50%(consine_similarity_score), 30%(weighted_salary) and 20%(weighted_rating). Lastly we created a dataframe in the decreasing order of total_score for each candidate.

## CONTRIBUTION 

In this notebook we have written 90% of the code on our own. Where ever we have used the code from any location we have cited the source.

## LICENSE 

This document is Licensed under the MIT license and is documented by Ayush Jain and Shweta Pathak.

https://opensource.org/licenses/MIT

MIT License

Copyright (c) 2019 Ayush Jain and Shweta Pathak

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.